In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [6]:
embeddings.embed_query("hello AI")

[-0.033388182520866394,
 0.03453972190618515,
 0.059474531561136246,
 0.05928609147667885,
 -0.0635354220867157,
 -0.06819586455821991,
 0.08823323994874954,
 0.03444080427289009,
 -0.03278516232967377,
 -0.015814989805221558,
 0.02098178118467331,
 -0.01834029331803322,
 -0.03983215242624283,
 -0.0804707482457161,
 -0.014469144865870476,
 0.0332648828625679,
 0.014259284362196922,
 -0.03404996916651726,
 -0.142915740609169,
 -0.023083344101905823,
 -0.021380102261900902,
 0.002633501309901476,
 -0.047292742878198624,
 -0.010752756148576736,
 -0.06866798549890518,
 0.031125057488679886,
 0.0759458914399147,
 0.0011283254716545343,
 0.011631987057626247,
 -0.03603919595479965,
 0.04483763128519058,
 0.018390750512480736,
 0.12672801315784454,
 -0.0013597895158454776,
 0.008206663653254509,
 0.06909968703985214,
 -0.08076353371143341,
 -0.05841314047574997,
 0.053754497319459915,
 0.026227595284581184,
 -0.006828607991337776,
 -0.056358352303504944,
 0.0032930178567767143,
 -0.0725017860

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [10]:
my_query="Narendra modi is prime minister of india?"

In [11]:
document_embedding=embeddings.embed_documents(documents)

In [ ]:
document_em1bedding

[[0.11998696625232697,
  -0.021302605047822,
  -0.04288087412714958,
  0.06645582616329193,
  -0.0643523558974266,
  -0.04424864798784256,
  0.022408470511436462,
  -0.049873027950525284,
  -0.023437663912773132,
  -0.03397207707166672,
  -0.0140480762347579,
  -0.06065931171178818,
  -0.003906761296093464,
  -0.017782077193260193,
  -0.047971006482839584,
  -0.0666816309094429,
  0.004103219136595726,
  -0.013092794455587864,
  0.0443977415561676,
  0.022350674495100975,
  0.009459568187594414,
  -0.02056453935801983,
  -0.00033560290466994047,
  -0.005685777403414249,
  0.05558697134256363,
  0.025123219937086105,
  -0.0028171155136078596,
  0.008759002201259136,
  0.003255249932408333,
  -0.015963444486260414,
  0.014263702556490898,
  -0.11220847815275192,
  0.0896855890750885,
  -0.031083744019269943,
  -0.02422383241355419,
  0.006152077112346888,
  0.08058709651231766,
  0.018250005319714546,
  0.05568312108516693,
  0.016702702268958092,
  0.01589597389101982,
  0.0003410273930

In [13]:
query_embedding=embeddings.embed_query(my_query)

In [14]:
len(query_embedding)

384

In [15]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756668, 0.3432456 , 0.81413237]])

In [17]:
from sklearn.metrics.pairwise import euclidean_distances

In [18]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848283, 1.14608416, 0.60970098]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [20]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [21]:
index=faiss.IndexFlatL2(384)


In [22]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000012AE99A6850> >

In [23]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [24]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['8e557fcc-b8d0-4e1f-8f38-74f176434561',
 'e0460b87-6a66-49ec-952c-c457ce6ec0e8',
 '1cc92c7f-4943-4f8c-a00f-f9cce9726e5f']

In [25]:
vector_store.index_to_docstore_id

{0: '8e557fcc-b8d0-4e1f-8f38-74f176434561',
 1: 'e0460b87-6a66-49ec-952c-c457ce6ec0e8',
 2: '1cc92c7f-4943-4f8c-a00f-f9cce9726e5f'}

In [26]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [27]:
results


[Document(id='e0460b87-6a66-49ec-952c-c457ce6ec0e8', metadata={}, page_content='AI is powerful'),
 Document(id='8e557fcc-b8d0-4e1f-8f38-74f176434561', metadata={}, page_content='AI is future'),
 Document(id='1cc92c7f-4943-4f8c-a00f-f9cce9726e5f', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [28]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [29]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [30]:
vector_store.add_documents(documents=documents)

['98f00299-b0bc-4c25-8d24-ac5db8814a83',
 '133c3c9f-6474-4657-8dee-3e4001a8726c',
 'aa5a1d82-d17a-41d8-916f-d067c6090f77',
 'ec722436-4693-4f98-8190-79aee8891403',
 'd35fbd15-c6d1-4bb4-9420-3b9a2e9be8c2',
 'd5f43ccf-6daa-4cd3-a466-17e10042daeb',
 '94d1eb7e-fe56-4584-b3ac-7b50d3a0fc11',
 '6a9799c1-dc94-447d-90bd-de0f512532a3',
 '6fc28ebe-5311-44a0-8576-9f3b0af8d04e',
 'd842ae9c-bafa-42bc-9ec3-55f03279b2db']

In [31]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='aa5a1d82-d17a-41d8-916f-d067c6090f77', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='6a9799c1-dc94-447d-90bd-de0f512532a3', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [33]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='aa5a1d82-d17a-41d8-916f-d067c6090f77', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='6a9799c1-dc94-447d-90bd-de0f512532a3', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='d842ae9c-bafa-42bc-9ec3-55f03279b2db', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='98f00299-b0bc-4c25-8d24-ac5db8814a83', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [34]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [35]:
result[0].metadata

{'source': 'news'}

In [36]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [37]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [49]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='e6ade172-ce20-4179-861e-147cf6e1c3d5', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a6e1aa11-4850-438c-acef-78be27abb03a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='85f4a46c-d5f6-4154-9685-1fdc071bf6cb', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [ ]:
inmemory(server)
ondisk(server)
cloud(yet to discuss)

In [38]:
vector_store.save_local("today's class faiss index")

In [39]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [55]:
new_vector_store.similarity_search("langchain")

[Document(id='e6ade172-ce20-4179-861e-147cf6e1c3d5', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a6e1aa11-4850-438c-acef-78be27abb03a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='b4c2983a-bcf2-4ba0-9182-4e68b530a9aa', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='f3fbd2f5-dd62-4bfc-b03f-02863a8bcc73', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [56]:
from langchain_community.document_loaders import PyPDFLoader

In [58]:
FILE_PATH=r"C:\Users\sunny\agenticai-2.0\data\llama2.pdf"

In [60]:
loader=PyPDFLoader(FILE_PATH)

In [64]:
len(loader.load())

77

In [67]:
pages=loader.load()

In [68]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [65]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [69]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [70]:
split_docs = splitter.split_documents(pages)

In [71]:
len(split_docs)

615

In [72]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [73]:
vector_store.add_documents(documents=split_docs)

['f025ac02-789e-4d7b-97c7-fa65b85352af',
 '46ea0e04-3cf2-4459-8dc6-89400d2c57d4',
 'e2b243d1-d093-4d4b-ad80-fa298891afb2',
 '341c5edd-fcdd-4e6c-8857-306d6f15a49e',
 'c000e1d7-872c-4652-aa83-182b4c105654',
 '708cefbf-2553-434f-b34f-7e907591ed68',
 '0df5e784-5312-4a75-9229-815d697288d8',
 '1ee8ab45-7b5a-44f6-a32d-f9f358d9aa6e',
 '22f5b5c9-723e-4813-83ec-84a3c6216ef2',
 'd0993f5c-e5d7-4b13-81be-35acbd90c0b2',
 '90c6ea44-68c1-4224-a8f5-d4f05c361b66',
 '9c883d10-6727-4354-af9a-582651e6b4a3',
 '34b92259-4597-40b9-99ee-b0f6205982fb',
 '8d2f4798-8d61-4bd8-913f-29e2c30a3043',
 '4c12a2e8-4e6a-4bf3-ad17-a282b2f66c54',
 '8764331a-f2bd-40f6-9c78-bac28bd57d71',
 'ee446a49-be69-4c4c-83a5-a7e4a5eb630b',
 '44b45c1f-3257-43ac-b0cb-8f9dd65dbc56',
 '79e762d3-40a1-4a91-a178-858cdcb8f783',
 'd572e2cc-f3eb-4cb0-b9a2-5b42fbe21e16',
 'c3f61d69-89cb-42a0-90f6-562bf85fe4e4',
 '61cabf5c-e7df-492c-80be-e6b862b1a00f',
 'e5c90ad1-b983-489e-98da-8c317100c487',
 '413740f3-8cac-4c83-bd6b-bb0fd4b74912',
 '978f81cd-4673-

In [78]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [79]:
retriever.invoke("what is llama model?")

[Document(id='978f81cd-4673-4b91-b8f2-a60d4de1550e', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\sunny\\agenticai-2.0\\data\\llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='89693474-8fe4-4edd-b381-6c2cec1a2a87', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywor

In [80]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [81]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\sunny\agenticai-2.0\env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [82]:
import pprint


In [83]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [84]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
context(retriever),prompt(hub),model(google),parser(langchain)

In [86]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [87]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [88]:
rag_chain.invoke("what is llama model?")

'Llama is a large language model developed by Meta.  It has various versions, including Llama 1 and Llama 2, with different parameter sizes.  Llama 2 is designed for commercial and research use and is available under a custom commercial license.'